In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_dir = "E:/DogBreedClassification/dogImages/train"
test_dir = "E:/DogBreedClassification/dogImages/test"
valid_dir = "E:/DogBreedClassification/dogImages/valid"


In [3]:
breed_freq = {}
for breed in os.listdir(train_dir):
    breed_path = os.path.join(train_dir,breed)
    if os.path.isdir(breed_path):
        freq = len(os.listdir(breed_path))
        breed_freq[breed] = freq

breed_freq_df = pd.DataFrame(list(breed_freq.items()), columns=['Breed', 'Frequency'])
print(breed_freq_df)

                               Breed  Frequency
0                  001.Affenpinscher         64
1                   002.Afghan_hound         58
2               003.Airedale_terrier         52
3                          004.Akita         63
4               005.Alaskan_malamute         77
..                               ...        ...
128              129.Tibetan_mastiff         48
129       130.Welsh_springer_spaniel         44
130  131.Wirehaired_pointing_griffon         30
131               132.Xoloitzcuintli         26
132            133.Yorkshire_terrier         30

[133 rows x 2 columns]


In [5]:
IMG_SIZE = (224, 224) # MobileNetV2 was trained on 224x224 images
BATCH_SIZE = 32

# Create an ImageDataGenerator for training data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create an ImageDataGenerator for validation and test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical' # For multi-class classification
)

valid_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Important for evaluation
)

# Get the number of classes (dog breeds)
num_classes = len(train_generator.class_indices)
print(f"Found {train_generator.samples} images belonging to {num_classes} classes in the training set.")
print(f"Found {valid_generator.samples} images in the validation set.")

Found 6678 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.
Found 6678 images belonging to 133 classes in the training set.
Found 835 images in the validation set.


In [33]:
# 1. Load the base model (MobileNetV2) with pre-trained ImageNet weights
base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,), # (224, 224, 3)
    include_top=False, # Exclude the final classification layer
    weights='imagenet'
)

# 2. Freeze the convolutional base
base_model.trainable = False

# 3. Add a custom classification head
# The head will take the features from the base model and learn to classify them
x = base_model.output
x = GlobalAveragePooling2D()(x) # Flattens the feature map
x = Dense(1024, activation='relu')(x) # A fully-connected layer
x = Dropout(0.5)(x) # Dropout for regularization to prevent overfitting
# The final output layer with softmax activation for classification
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and the new head to create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# 4. Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print a summary of the model architecture
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,706,053 (14.14 MB)

 Trainable params: 1,448,069 (5.52 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [34]:
# Define the number of epochs
EPOCHS = 10

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)

# Save the trained model for future use
model.save('dog_breed_classifier_mobilenetv2.h5')
print("Model saved successfully!")

e:\DogBreedClassification\my_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 152s 712ms/step - accuracy: 0.3959 - loss: 2.5757 - val_accuracy: 0.7365 - val_loss: 0.9093
Epoch 2/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 154s 735ms/step - accuracy: 0.5927 - loss: 1.4307 - val_accuracy: 0.7689 - val_loss: 0.7279
Epoch 3/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 151s 720ms/step - accuracy: 0.6384 - loss: 1.2324 - val_accuracy: 0.7892 - val_loss: 0.6575
Epoch 4/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 151s 720ms/step - accuracy: 0.6664 - loss: 1.1197 - val_accuracy: 0.8012 - val_loss: 0.6332
Epoch 5/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 150s 717ms/step - accuracy: 0.6954 - loss: 1.0351 - val_accuracy: 0.7916 - val_loss: 0.6400
Epoch 6/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 150s 718ms/step - accuracy: 0.6992 - loss: 0.9953 - val_accuracy: 0.8048 - val_loss: 0.6399
Epoch 7/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 150s 718ms/step - accuracy: 0.7096 - loss: 0.9620 - val_accuracy: 0.7796 - val_loss: 0.6654
Epoch 8/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 152s 728ms/step - accuracy: 0.7260 -

Model saved successfully!


In [24]:
from PIL import Image
import os

def is_image_valid(file_path):
    try:
        with Image.open(file_path) as img:
            img.load()  # Fully load image data
            # img.convert('RGB')  # Optional: simulate preprocessing
        return True
    except Exception as e:
        print(f"Corrupt image: {file_path} — {e}")
        return False

# Scan all images
for folder in dirs_to_check:
    for subdir, _, files in os.walk(folder):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                file_path = os.path.join(subdir, file)
                if not is_image_valid(file_path):
                    # Optionally delete or move the file
                    # os.remove(file_path)
                    pass


Corrupt image: dogImages\train\098.Leonberger\Leonberger_06571.jpg — image file is truncated (148 bytes not processed)


In [6]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input

base_model = ResNet50V2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model
base_model.trainable = False

# Add the custom classification head (this part is the same)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

EPOCHS = 10

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)

model.save('dog_breed_classifier_resnet50v2.h5')
print("ResNet50V2 model saved successfully!")

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 25,799,301 (98.42 MB)

 Trainable params: 2,234,501 (8.52 MB)

...

e:\DogBreedClassification\my_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2774 - loss: 3.4991

e:\DogBreedClassification\my_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


209/209 ━━━━━━━━━━━━━━━━━━━━ 636s 3s/step - accuracy: 0.4033 - loss: 2.5389 - val_accuracy: 0.7126 - val_loss: 0.9172
Epoch 2/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 506s 2s/step - accuracy: 0.5818 - loss: 1.5116 - val_accuracy: 0.7401 - val_loss: 0.7920
Epoch 3/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 428s 2s/step - accuracy: 0.6289 - loss: 1.2996 - val_accuracy: 0.7701 - val_loss: 0.7420
Epoch 4/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 307s 1s/step - accuracy: 0.6574 - loss: 1.2013 - val_accuracy: 0.7796 - val_loss: 0.6605
Epoch 5/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 331s 2s/step - accuracy: 0.6629 - loss: 1.1530 - val_accuracy: 0.7868 - val_loss: 0.7097
Epoch 6/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.6827 - loss: 1.0874 - val_accuracy: 0.7928 - val_loss: 0.6861
Epoch 7/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.6902 - loss: 1.0434 - val_accuracy: 0.7880 - val_loss: 0.6858
Epoch 8/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.7034 - loss: 1.0243 - val_accuracy: 0.790

ResNet50V2 model saved successfully!


In [7]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Load InceptionV3 base model
base_model = InceptionV3(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base model
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
EPOCHS = 10
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=valid_generator
)

# Save
model.save('dog_breed_classifier_inceptionv3.h5')
print("InceptionV3 model saved successfully!")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d_3[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 24,037,285 (91.69 MB)

 Trainable params: 2,234,501 (8.52 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - accuracy: 0.4588 - loss: 2.4168 - val_accuracy: 0.7820 - val_loss: 0.7790
Epoch 2/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 247s 1s/step - accuracy: 0.6035 - loss: 1.4755 - val_accuracy: 0.8072 - val_loss: 0.6597
Epoch 3/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.6330 - loss: 1.3156 - val_accuracy: 0.8228 - val_loss: 0.5808
Epoch 4/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - accuracy: 0.6530 - loss: 1.2358 - val_accuracy: 0.8180 - val_loss: 0.5781
Epoch 5/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 257s 1s/step - accuracy: 0.6691 - loss: 1.1814 - val_accuracy: 0.8287 - val_loss: 0.5695
Epoch 6/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.6731 - loss: 1.1576 - val_accuracy: 0.8251 - val_loss: 0.5722
Epoch 7/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.6831 - loss: 1.1116 - val_accuracy: 0.8287 - val_loss: 0.5540
Epoch 8/10
209/209 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.6909 - loss: 1.0661 - val_accu

InceptionV3 model saved successfully!
